In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import getfundinfo
import getfundshareholds
import time
import datetime

# 数据采集

## 提取基金主页信息
通过爬取天天基金网混合基金网站，截取近2年累计收益率前100名的基金数据

In [3]:
url = 'http://fund.eastmoney.com/trade/hh.html'
fund_page = getfundinfo.Getfund(url=url,sortby='近2年')
td_list,f_name_urls,table_heads = fund_page.get_fund_info()
td_records = fund_page.split_dataset(td_list,15)

In [4]:
# 合并为一个DataFrame
f_df = pd.DataFrame(data=td_records,columns=table_heads)
f_df['基金链接'] = f_name_urls
f_df.head()

,基金代码,基金名称,单位净值|日期,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,手续费|起购金额,操作,基金链接
0,161903,万家行业优选混合(LOF),2.105611-24,-0.01%,1.01%,9.01%,11.49%,37.70%,102.12%,246.29%,203.74%,87.61%,942.67%,0.15%1.50%100元,买入定投,http://fund.eastmoney.com/161903.html
1,100060,富国高新技术产业混合,4.409011-24,-0.23%,1.17%,10.11%,14.40%,72.16%,111.67%,233.08%,155.09%,87.54%,515.00%,0.15%1.50%100元,买入定投,http://fund.eastmoney.com/100060.html
2,005911,广发双擎升级混合A,3.141811-24,-0.36%,0.61%,5.81%,-1.80%,24.06%,70.98%,229.88%,---,51.36%,229.81%,0.15%1.50%100元,买入定投,http://fund.eastmoney.com/005911.html
3,001679,前海开源中国稀缺资产混合A,2.618011-24,0.69%,2.15%,6.55%,4.22%,49.34%,92.08%,225.62%,217.33%,82.95%,161.80%,0.15%1.50%100元,买入定投,http://fund.eastmoney.com/001679.html
4,002079,前海开源中国稀缺资产混合C,2.741011-24,0.70%,2.16%,6.53%,4.18%,49.29%,91.95%,225.15%,220.96%,82.86%,174.10%,0.15%1.50%100元,买入定投,http://fund.eastmoney.com/002079.html


In [5]:
yesterday = (datetime.date.today() + datetime.timedelta(-1)).strftime('%Y-%m-%d')
yesterday

'2020-11-24'

In [6]:
# 将基金基本信息储存至本地
f_df.to_csv('fundInfo_'+yesterday+'.csv',index=False,encoding='gbk')

## 提取基金股票持仓数据
提取`f_df`中每一只基金对应年份对应季度的股票重仓数据，需要包含的数据变量有：

- 基金代码
- 股票代码
- 股票名称
- 持仓占比
- 持仓截止日期

> 根据基金链接和年份获取该基金对应年份的所有持仓数据，更多持仓数据的链接可以通过基金链接来获取，如：
**万家行业优选混合(LOF)**
- 基金链接：http://fund.eastmoney.com/161903.html 
- 更多持仓数据链接：http://fundf10.eastmoney.com/ccmx_161903.html  
更多持仓数据链接是在基金链接的`基金代码`前面增加了`ccmx_`，在`fund`后面增加了`f10`

In [8]:
f_df['更多持仓信息链接']= f_df['基金链接'].str.replace('http://fund.eastmoney.com/','http://fundf10.eastmoney.com/').apply(lambda x:x[:-11] + 'ccmx_' + x[-11:])
f_df['更多持仓信息链接'].head(2)

0    http://fundf10.eastmoney.com/ccmx_161903.html
1    http://fundf10.eastmoney.com/ccmx_100060.html
Name: 更多持仓信息链接, dtype: object

In [9]:
fund_cc_urls = f_df['更多持仓信息链接']
unsucessful_fundlid = []


start_time = time.time()
for i, url in enumerate(fund_cc_urls):

    # 出现网络加载速度慢，过了sleeptime仍然没有加载成功，则忽略，继续爬取下一个基金信息
    try:
        fund_sh = getfundshareholds.Getshareholds(url,year=2020)
        if (i > 0):
            allfund_shs = np.append(allfund_shs,fund_sh.get_fund_shareholds(),axis=0)
        else:
            allfund_shs = fund_sh.get_fund_shareholds()
        print('第{}只基金持仓数据已获取完成'.format(i+1))
    except:
        unsucessful_fundlid.append(i+1)
        print('第{}只基金持仓数据没有获取完成'.format(i+1))
        continue

    time.sleep(3) # 每一只基金爬取完成休息5秒
end_time = time.time()
print("运行时间共计：{}秒".format(round(end_time - start_time,2)))

第1只基金持仓数据已获取完成
第2只基金持仓数据已获取完成
第3只基金持仓数据已获取完成
第4只基金持仓数据已获取完成
第5只基金持仓数据已获取完成
第6只基金持仓数据已获取完成
第7只基金持仓数据已获取完成
第8只基金持仓数据已获取完成
第9只基金持仓数据已获取完成
第10只基金持仓数据已获取完成
第11只基金持仓数据已获取完成
第12只基金持仓数据已获取完成
第13只基金持仓数据已获取完成
第14只基金持仓数据已获取完成
第15只基金持仓数据已获取完成
第16只基金持仓数据已获取完成
第17只基金持仓数据已获取完成
第18只基金持仓数据已获取完成
第19只基金持仓数据已获取完成
第20只基金持仓数据已获取完成
第21只基金持仓数据已获取完成
第22只基金持仓数据已获取完成
第23只基金持仓数据已获取完成
第24只基金持仓数据已获取完成
第25只基金持仓数据已获取完成
第26只基金持仓数据已获取完成
第27只基金持仓数据已获取完成
第28只基金持仓数据已获取完成
第29只基金持仓数据已获取完成
第30只基金持仓数据已获取完成
第31只基金持仓数据已获取完成
第32只基金持仓数据已获取完成
第33只基金持仓数据已获取完成
第34只基金持仓数据已获取完成
第35只基金持仓数据已获取完成
第36只基金持仓数据已获取完成
第37只基金持仓数据已获取完成
第38只基金持仓数据已获取完成
第39只基金持仓数据已获取完成
第40只基金持仓数据已获取完成
第41只基金持仓数据已获取完成
第42只基金持仓数据已获取完成
第43只基金持仓数据已获取完成
第44只基金持仓数据已获取完成
第45只基金持仓数据已获取完成
第46只基金持仓数据已获取完成
第47只基金持仓数据已获取完成
第48只基金持仓数据已获取完成
第49只基金持仓数据已获取完成
第50只基金持仓数据已获取完成
第51只基金持仓数据已获取完成
第52只基金持仓数据已获取完成
第53只基金持仓数据已获取完成
第54只基金持仓数据已获取完成
第55只基金持仓数据已获取完成
第56只基金持仓数据已获取完成
第57只基金持仓数据已获取完成
第58只基金持仓数据已获取完成
第59只基金持仓数据已获取完成
第60只基金持仓数据已获取完成
第61只基金持仓数据已获取完成
第62只基金持仓数据已获取完成
第63只基金持仓数据已获取完成
第

In [10]:
# 查看是否存在部分基金没有成功获取其持仓数据
unsucessful_fundlid

[]

In [11]:
# 获取完成后需要用DataFrame封装
column_names = ['基金代码', '重仓股票代码', '股票名称', '持仓占比', '持仓截止日期']
allfund_shs_df = pd.DataFrame(data=allfund_shs,columns=column_names)
allfund_shs_df.head()

,基金代码,重仓股票代码,股票名称,持仓占比,持仓截止日期
0,161903,002594,比亚迪,9.26%,2020-09-30
1,161903,603986,兆易创新,8.75%,2020-09-30
2,161903,600588,用友网络,8.70%,2020-09-30
3,161903,300454,深信服,8.50%,2020-09-30
4,161903,300676,华大基因,6.33%,2020-09-30


In [12]:
# 储存爬取的基金持仓数据
recent_enddate = allfund_shs_df['持仓截止日期'].unique()[0]
allfund_shs_df.to_csv('top100fund_shs_'+str(recent_enddate)+'.csv',encoding='gbk',index=False)